# Bluesky & ophyd: Lesson 1 - EpicsScaler

## Starting this session in a Jupyter notebook

This session was started from the linux command line:

```
jemian@otz .../JEMIAN/Documents $ /APSshare/anaconda3/BlueSky/bin/jupyter-notebook 
```

This command produced the following console output and then started my default web browser with a one-time-token-authenticated connection to the Jupyter Notebook server (still running in the console):

```
[I 15:16:57.546 NotebookApp] Serving notebooks from local directory: /home/oxygen18/JEMIAN/Documents
[I 15:16:57.546 NotebookApp] 0 active kernels
[I 15:16:57.546 NotebookApp] The Jupyter Notebook is running at:
[I 15:16:57.546 NotebookApp] http://localhost:8888/?token=e6a7584762c731a7c64f8f71246b3e616d779f7b4852c9d9
[I 15:16:57.546 NotebookApp] Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
[C 15:16:57.547 NotebookApp] 
    
    Copy/paste this URL into your browser when you connect for the first time,
    to login with a token:
        http://localhost:8888/?token=e6a7584762c731a7c64f8f71246b3e616d779f7b4852c9d9
[I 15:17:00.863 NotebookApp] Accepting one-time-token-authenticated connection from ::1
```

Next, found the **New** drop-down menu button (top right, below the **Lougout** button) and selected *Python 3* to start a new notebook page using a Python 3 shell (the only kind available here).

Finally, from the **File** menu in the jupyter notebook (in the browser), selected *Rename ...* to save the *Untitled* notebook with the name *lesson1* (default extension is `.ipynb`).

## Connect an EPICS scaler

I have a synApps (v5.8) *XXX*-style IOC with the prefix `gov:`.  It has a [scaler](https://htmlpreview.github.io/?https://raw.githubusercontent.com/epics-modules/std/R3-4-1/documentation/scalerRecord.html), 16 soft channel [motor](https://github.com/epics-modules/motor)s, and some other support we'll ignore in lesson 1.

The scaler is `gov:scaler1`.  We'll connect to that first.  To make the connection, we need to import the [`EpicsScaler`](http://nsls-ii.github.io/ophyd/builtin-devices.html#epicsscaler) device from the [`ophyd`](http://nsls-ii.github.io/ophyd/) library.

In [1]:
from ophyd import EpicsScaler

Now we can create the scaler object we'll use as a detector.

In [2]:
scaler = EpicsScaler("gov:scaler1", name="scaler")

Let's test that connection by asking the scaler to read it's values from EPICS.

In [3]:
scaler.read()

OrderedDict([('scaler_channels_chan1',
              {'value': 0.0, 'timestamp': 1532469272.141368}),
             ('scaler_channels_chan2',
              {'value': 0.0, 'timestamp': 1532469272.141368}),
             ('scaler_channels_chan3',
              {'value': 0.0, 'timestamp': 1532469272.141368}),
             ('scaler_channels_chan4',
              {'value': 0.0, 'timestamp': 1532469272.141368}),
             ('scaler_channels_chan5',
              {'value': 0.0, 'timestamp': 1532469272.141368}),
             ('scaler_channels_chan6',
              {'value': 0.0, 'timestamp': 1532469272.141368}),
             ('scaler_channels_chan7',
              {'value': 0.0, 'timestamp': 1532469272.141368}),
             ('scaler_channels_chan8',
              {'value': 0.0, 'timestamp': 1532469272.141368}),
             ('scaler_channels_chan9',
              {'value': 0.0, 'timestamp': 1532469272.141368}),
             ('scaler_channels_chan10',
              {'value': 0.0, 'timestamp': 

By default, the ophyd scaler object is prepared to connect to 32 channels from the EPICS scaler.  The names of the channels can be displayed, but it takes more work.  We'll do this with a loop without explaining the details.

In [4]:
for nm in scaler.names.configuration_attrs:
    ch = scaler.names.__getattribute__(nm)
    print(nm, ch.value)

name1 clock
name2 I0
name3 I
name4 diode
name5 
name6 
name7 
name8 
name9 
name10 
name11 
name12 
name13 
name14 
name15 
name16 
name17 
name18 
name19 
name20 
name21 
name22 
name23 
name24 
name25 
name26 
name27 
name28 
name29 
name30 
name31 
name32 


Our scaler only has a few channels in use (that is, channels where the name of the channel has been specified in the GUI screen).  Let's focus on just those channels.

To read just the *named* channels, we'll load an additional support library.  (Loading additional support as we need it will be a common theme in these lessons.  Rather than loading all the libraries first, as is common in python code files, we'll load support code as the need arises.)

The new support code is `use_EPICS_scaler_channels` from [`APS_BlueSky_tools.devices`](http://aps-bluesky-tools.readthedocs.io/en/latest/source_code/devices.html#APS_BlueSky_tools.devices.use_EPICS_scaler_channels).

In [5]:
from APS_BlueSky_tools.devices import use_EPICS_scaler_channels

That is a utility function that will modify our `scaler` so it only reports the named channels.  Now the report is succinct.

In [6]:
use_EPICS_scaler_channels(scaler)
scaler.read()

OrderedDict([('scaler_channels_chan1',
              {'value': 0.0, 'timestamp': 1532469272.141368}),
             ('scaler_channels_chan2',
              {'value': 0.0, 'timestamp': 1532469272.141368}),
             ('scaler_channels_chan3',
              {'value': 0.0, 'timestamp': 1532469272.141368}),
             ('scaler_channels_chan4',
              {'value': 0.0, 'timestamp': 1532469272.141368})])

If you want to change the name fields on any of the scaler channels from the command line, follow this example.

In [7]:
scaler.names.name7.put("roi1")
scaler.names.name4.put("scint")

Then print out the names of only the *named* channels.

In [8]:
for nm in scaler.names.configuration_attrs:
    ch = scaler.names.__getattribute__(nm)
    if len(ch.value) > 0:
        print(nm, ch.value)

name1 clock
name2 I0
name3 I
name4 scint
name7 roi1


Then, update the `scaler` object for these channels.

In [9]:
use_EPICS_scaler_channels(scaler)
scaler.read()

OrderedDict([('scaler_channels_chan1',
              {'value': 0.0, 'timestamp': 1532469272.141368}),
             ('scaler_channels_chan2',
              {'value': 0.0, 'timestamp': 1532469272.141368}),
             ('scaler_channels_chan3',
              {'value': 0.0, 'timestamp': 1532469272.141368}),
             ('scaler_channels_chan4',
              {'value': 0.0, 'timestamp': 1532469272.141368}),
             ('scaler_channels_chan7',
              {'value': 0.0, 'timestamp': 1532469272.141368})])

## Get, then Set the count time on the scaler

The preset counting time (`.TP`) is one of the many fields of the EPICS scaler record.  It is available as `scaler.preset_time` in this scaler object.  We can inspect this:

In [10]:
scaler.preset_time

EpicsSignal(read_pv='gov:scaler1.TP', name='scaler_preset_time', parent='scaler', value=3.0, timestamp=1532469272.141368, tolerance=0.001, pv_kw={}, auto_monitor=False, string=False, write_pv='gov:scaler1.TP', limits=False, put_complete=False)

Since `scaler.preset_time` is an [`EpicsSignal`](https://github.com/NSLS-II/ophyd/blob/master/ophyd/signal.py#L647), ([Signal](http://nsls-ii.github.io/ophyd/signals.html) is a fundamental ophyd structure), we print its `.value`.

In [11]:
scaler.preset_time.value

3.0

Set the scaler to count for 1.5 seconds once it is told to count.  We'll count it next.

In [12]:
scaler.preset_time.put(1.5)

Note, to tell the scaler to count, we put a 1 to its `.CNT` field.

In [13]:
scaler.count.put(1)

Now, read the scaler again.  This scaler is not very interesting since it is not connected to any hardware.  But, the timestamps have changed.

In [14]:
scaler.read()

OrderedDict([('scaler_channels_chan1',
              {'value': 0.0, 'timestamp': 1532469272.141368}),
             ('scaler_channels_chan2',
              {'value': 0.0, 'timestamp': 1532469272.141368}),
             ('scaler_channels_chan3',
              {'value': 0.0, 'timestamp': 1532469272.141368}),
             ('scaler_channels_chan4',
              {'value': 0.0, 'timestamp': 1532469272.141368}),
             ('scaler_channels_chan7',
              {'value': 0.0, 'timestamp': 1532469272.141368})])

## Use Bluesky to count the scaler

So far, we have used ophyd to talk with the EPICS scaler.  Let's start to use Bluesky.  We need to load support to start the Bluesky [*RunEngine*](http://nsls-ii.github.io/bluesky/tutorial.html#the-runengine).  For now, we'll use the most basic configuration (does not save data anywhere).  

The job of the RunEngine is to process command messages (our data acquisition commands) and to output documents (the data to be acquired).  Initially, we'll use predefined data acquisition sequences (called [*plans*](http://nsls-ii.github.io/bluesky/plans.html)) and ignore the [documents](http://nsls-ii.github.io/bluesky/documents.html).

In [15]:
from bluesky import RunEngine
RE = RunEngine({})

Next, we'll load a library with many [predefined plans](http://nsls-ii.github.io/bluesky/plans.html#summary).  The first plan we want to use is [`count()`](http://nsls-ii.github.io/bluesky/generated/bluesky.plans.count.html#bluesky.plans.count).  By the way, we'll use a python feature to rename `bluesky.plans` to the shorter `bp` since it will be used a lot.

In [16]:
import bluesky.plans as bp

Now we can count the `scaler` by submitting the `bp.count` plan to the RunEngine instance `RE`.  The `bp.count()` plan takes one argument, a *list* of countable objects (just in case you want to count more than one detector at the same time).  Our list has only one object: `scaler`

In [17]:
RE(bp.count([scaler]))

('f333097a-7af8-496f-aedd-884c86ddc452',)

The final result from submitting a plan to the RunEngine is an identifier of the sequence of documents.  Since we did not capture that sequence, we can't view it or access it in any way.  Let's *fix* that next.

### Report about the documents emitted from the RunEngine

To display information about the documents emitted from the RunEngine, we need to make a function that will receive the documents.  

In Bluesky terms, this type of function is a *callback*.  It takes two arguments.  The first argument is a `str` that tells what kind of document is coming, the second is a python dictionary with the document's contents.  We'll start by printing summary information.  We then submit the plan again with the name of our callback function as a second argument (not to `bp.count` but as a second argument to `RE()`).

In [18]:
def myCallback(key, doc):
    print(key, len(doc))

In [19]:
RE(bp.count([scaler]), myCallback)

start 10
descriptor 8
event 7
stop 6


('af62c72f-2a09-4055-b09e-8a87018d4070',)

So, there were four documents.  See http://nsls-ii.github.io/bluesky/documents.html for the details of each.  Let's extend our callback to print the details of each.

In [20]:
def myCallback(key, doc):
    print(key, len(doc))
    for k, v in doc.items():
        print("\t", k, v)
    print("~~~~~~~~~~~~~~~~~")

In [21]:
RE(bp.count([scaler]), myCallback)

start 10
	 uid 745077f0-d34d-4e9e-9922-6f5c0cedd856
	 time 1532469311.0539966
	 hints {'dimensions': [(('time',), 'primary')]}
	 plan_args {'detectors': ["EpicsScaler(prefix='gov:scaler1', name='scaler', read_attrs=['channels', 'channels.chan1', 'channels.chan2', 'channels.chan3', 'channels.chan4', 'channels.chan7', 'names', 'presets', 'gates'], configuration_attrs=['count_mode', 'delay', 'auto_count_delay', 'channels', 'names', 'names.name1', 'names.name2', 'names.name3', 'names.name4', 'names.name5', 'names.name6', 'names.name7', 'names.name8', 'names.name9', 'names.name10', 'names.name11', 'names.name12', 'names.name13', 'names.name14', 'names.name15', 'names.name16', 'names.name17', 'names.name18', 'names.name19', 'names.name20', 'names.name21', 'names.name22', 'names.name23', 'names.name24', 'names.name25', 'names.name26', 'names.name27', 'names.name28', 'names.name29', 'names.name30', 'names.name31', 'names.name32', 'time', 'freq', 'preset_time', 'auto_count_time', 'presets', 'ga

('745077f0-d34d-4e9e-9922-6f5c0cedd856',)

With the details, we see more of what is happening.  

The first document is a `start` and it says what is happening and how it will be identified.  The identifier is a [UUID](https://docs.python.org/3.6/library/uuid.html) (specifically, a [uuid4](https://docs.python.org/3.6/library/uuid.html#uuid.uuid4)).  This looks tedious and random.  The one good thing is that these are unique and often can be referred to by the first few characters (6-8 are enough to be *probably* unique).  The UUID from the `start` document is what the RunEngine reports when the scan stops.

The second document is a `descriptor` and it says what, exactly, will be measured.  It also includes a reference to `start`'s uuid.  We can follow the chain back to `start` this way.

The third document is an `event` and it provides the data reading when the scaler was counted.  The data is read according to the `descriptor` document (just actual values for the *named* channels).

The last document is a `stop` document that describes how things ended.  It also says there was only one stream of event documents named *primary*.

Let's repeat that plan, this time asking to `count` the scaler three times.  This is the `num=3` addition to the call to `bp.count()`.  What's tricky now is keeping track of the parentheses and square braces.

In [22]:
RE(bp.count([scaler], num=3), myCallback)

start 10
	 uid b7e9f118-5eb5-477f-9de4-ddca5c8b47a2
	 time 1532469312.8402565
	 hints {'dimensions': [(('time',), 'primary')]}
	 plan_args {'detectors': ["EpicsScaler(prefix='gov:scaler1', name='scaler', read_attrs=['channels', 'channels.chan1', 'channels.chan2', 'channels.chan3', 'channels.chan4', 'channels.chan7', 'names', 'presets', 'gates'], configuration_attrs=['count_mode', 'delay', 'auto_count_delay', 'channels', 'names', 'names.name1', 'names.name2', 'names.name3', 'names.name4', 'names.name5', 'names.name6', 'names.name7', 'names.name8', 'names.name9', 'names.name10', 'names.name11', 'names.name12', 'names.name13', 'names.name14', 'names.name15', 'names.name16', 'names.name17', 'names.name18', 'names.name19', 'names.name20', 'names.name21', 'names.name22', 'names.name23', 'names.name24', 'names.name25', 'names.name26', 'names.name27', 'names.name28', 'names.name29', 'names.name30', 'names.name31', 'names.name32', 'time', 'freq', 'preset_time', 'auto_count_time', 'presets', 'ga

event 7
	 descriptor 02cf2359-106f-4fb2-bd73-7d054dcc5b7f
	 time 1532469316.4139385
	 data {'scaler_channels_chan1': 16000000.0, 'scaler_channels_chan2': 6.0, 'scaler_channels_chan3': 6.0, 'scaler_channels_chan4': 7.0, 'scaler_channels_chan7': 9.0}
	 timestamps {'scaler_channels_chan1': 1532469316.408403, 'scaler_channels_chan2': 1532469316.408403, 'scaler_channels_chan3': 1532469316.408403, 'scaler_channels_chan4': 1532469316.408403, 'scaler_channels_chan7': 1532469316.408403}
	 seq_num 2
	 uid c2a6de4c-0463-48fd-bc3c-f4dc379f6c5d
	 filled {}
~~~~~~~~~~~~~~~~~
event 7
	 descriptor 02cf2359-106f-4fb2-bd73-7d054dcc5b7f
	 time 1532469318.0446608
	 data {'scaler_channels_chan1': 16000000.0, 'scaler_channels_chan2': 7.0, 'scaler_channels_chan3': 8.0, 'scaler_channels_chan4': 7.0, 'scaler_channels_chan7': 8.0}
	 timestamps {'scaler_channels_chan1': 1532469318.039402, 'scaler_channels_chan2': 1532469318.039402, 'scaler_channels_chan3': 1532469318.039402, 'scaler_channels_chan4': 1532469318.0

('b7e9f118-5eb5-477f-9de4-ddca5c8b47a2',)

Mostly the same but now there are three `event` documents.

# Summary

We'll show the code so far as a proper python program now:

```
#!/APSshare/anaconda3/BlueSky/bin/python

"lesson 1: EpicsScaler"

from ophyd import EpicsScaler
from bluesky import RunEngine
import bluesky.plans as bp
from APS_BlueSky_tools.devices import use_EPICS_scaler_channels


def myCallback(key, doc):
    print(key, len(doc))
    for k, v in doc.items():
        print("\t", k, v)
    print("~~~~~~~~~~~~~~~~~")

def printScalerNames(scaler):
    for nm in scaler.names.configuration_attrs:
        ch = scaler.names.__getattribute__(nm)
        if len(ch.value) > 0:
            print(nm, ch.value)


RE = RunEngine({})

scaler = EpicsScaler("gov:scaler1", name="scaler")
scaler.preset_time.put(1.5)
print(scaler.preset_time.value)

scaler.names.name7.put("roi1")
scaler.names.name4.put("scint")
printScalerNames(scaler)

use_EPICS_scaler_channels(scaler)
print(scaler.read())
print(RE(bp.count([scaler], num=3), myCallback))
```


Here is the output when run from the console:

```
jemian@otz /tmp $ ./lesson1.py 
1.5
name1 clock
name2 I0
name3 I
name4 scint
name7 roi1
OrderedDict([('scaler_channels_chan1', {'value': 0.0, 'timestamp': 1532469318.054875}), ('scaler_channels_chan2', {'value': 0.0, 'timestamp': 1532469318.054875}), ('scaler_channels_chan3', {'value': 0.0, 'timestamp': 1532469318.054875}), ('scaler_channels_chan4', {'value': 0.0, 'timestamp': 1532469318.054875}), ('scaler_channels_chan7', {'value': 0.0, 'timestamp': 1532469318.054875})])
start 10
	 uid 38f77323-57e9-443e-8114-871f85347610
	 time 1532469886.0000541
	 num_points 3
	 plan_name count
	 plan_args {'detectors': ["EpicsScaler(prefix='gov:scaler1', name='scaler', read_attrs=['channels', 'channels.chan1', 'channels.chan2', 'channels.chan3', 'channels.chan4', 'channels.chan7', 'names', 'presets', 'gates'], configuration_attrs=['count_mode', 'delay', 'auto_count_delay', 'channels', 'names', 'names.name1', 'names.name2', 'names.name3', 'names.name4', 'names.name5', 'names.name6', 'names.name7', 'names.name8', 'names.name9', 'names.name10', 'names.name11', 'names.name12', 'names.name13', 'names.name14', 'names.name15', 'names.name16', 'names.name17', 'names.name18', 'names.name19', 'names.name20', 'names.name21', 'names.name22', 'names.name23', 'names.name24', 'names.name25', 'names.name26', 'names.name27', 'names.name28', 'names.name29', 'names.name30', 'names.name31', 'names.name32', 'time', 'freq', 'preset_time', 'auto_count_time', 'presets', 'gates', 'update_rate', 'auto_count_update_rate', 'egu'])"], 'num': 3}
	 hints {'dimensions': [(('time',), 'primary')]}
	 num_intervals 2
	 plan_type generator
	 scan_id 1
	 detectors ['scaler']
~~~~~~~~~~~~~~~~~
descriptor 8
	 run_start 38f77323-57e9-443e-8114-871f85347610
	 time 1532469887.825319
	 data_keys {'scaler_channels_chan1': {'source': 'PV:gov:scaler1.S1', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0, 'object_name': 'scaler'}, 'scaler_channels_chan2': {'source': 'PV:gov:scaler1.S2', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0, 'object_name': 'scaler'}, 'scaler_channels_chan3': {'source': 'PV:gov:scaler1.S3', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0, 'object_name': 'scaler'}, 'scaler_channels_chan4': {'source': 'PV:gov:scaler1.S4', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0, 'object_name': 'scaler'}, 'scaler_channels_chan7': {'source': 'PV:gov:scaler1.S7', 'dtype': 'number', 'shape': [], 'precision': 0, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0, 'object_name': 'scaler'}}
	 uid cd11f35c-0811-4533-ab72-68f68d950fa0
	 configuration {'scaler': {'data': {'scaler_count_mode': 'OneShot', 'scaler_delay': 0.0, 'scaler_auto_count_delay': 1.0, 'scaler_names_name1': 'clock', 'scaler_names_name2': 'I0', 'scaler_names_name3': 'I', 'scaler_names_name4': 'scint', 'scaler_names_name5': '', 'scaler_names_name6': '', 'scaler_names_name7': 'roi1', 'scaler_names_name8': '', 'scaler_names_name9': '', 'scaler_names_name10': '', 'scaler_names_name11': '', 'scaler_names_name12': '', 'scaler_names_name13': '', 'scaler_names_name14': '', 'scaler_names_name15': '', 'scaler_names_name16': '', 'scaler_names_name17': '', 'scaler_names_name18': '', 'scaler_names_name19': '', 'scaler_names_name20': '', 'scaler_names_name21': '', 'scaler_names_name22': '', 'scaler_names_name23': '', 'scaler_names_name24': '', 'scaler_names_name25': '', 'scaler_names_name26': '', 'scaler_names_name27': '', 'scaler_names_name28': '', 'scaler_names_name29': '', 'scaler_names_name30': '', 'scaler_names_name31': '', 'scaler_names_name32': '', 'scaler_time': 1.6, 'scaler_freq': 10000000.0, 'scaler_preset_time': 1.5, 'scaler_auto_count_time': 1.0, 'scaler_update_rate': 10.0, 'scaler_auto_count_update_rate': 10.0, 'scaler_egu': ''}, 'timestamps': {'scaler_count_mode': 1532469887.717194, 'scaler_delay': 1532469887.717194, 'scaler_auto_count_delay': 1532469887.717194, 'scaler_names_name1': 1532469887.717194, 'scaler_names_name2': 1532469887.717194, 'scaler_names_name3': 1532469887.717194, 'scaler_names_name4': 1532469887.717194, 'scaler_names_name5': 1532469887.717194, 'scaler_names_name6': 1532469887.717194, 'scaler_names_name7': 1532469887.717194, 'scaler_names_name8': 1532469887.717194, 'scaler_names_name9': 1532469887.717194, 'scaler_names_name10': 1532469887.717194, 'scaler_names_name11': 1532469887.717194, 'scaler_names_name12': 1532469887.717194, 'scaler_names_name13': 1532469887.717194, 'scaler_names_name14': 1532469887.717194, 'scaler_names_name15': 1532469887.717194, 'scaler_names_name16': 1532469887.717194, 'scaler_names_name17': 1532469887.717194, 'scaler_names_name18': 1532469887.717194, 'scaler_names_name19': 1532469887.717194, 'scaler_names_name20': 1532469887.717194, 'scaler_names_name21': 1532469887.717194, 'scaler_names_name22': 1532469887.717194, 'scaler_names_name23': 1532469887.717194, 'scaler_names_name24': 1532469887.717194, 'scaler_names_name25': 1532469887.717194, 'scaler_names_name26': 1532469887.717194, 'scaler_names_name27': 1532469887.717194, 'scaler_names_name28': 1532469887.717194, 'scaler_names_name29': 1532469887.717194, 'scaler_names_name30': 1532469887.717194, 'scaler_names_name31': 1532469887.717194, 'scaler_names_name32': 1532469887.717194, 'scaler_time': 1532469887.717194, 'scaler_freq': 1532469887.717194, 'scaler_preset_time': 1532469887.717194, 'scaler_auto_count_time': 1532469887.717194, 'scaler_update_rate': 1532469887.717194, 'scaler_auto_count_update_rate': 1532469887.717194, 'scaler_egu': 1532469887.717194}, 'data_keys': OrderedDict([('scaler_count_mode', {'source': 'PV:gov:scaler1.CONT', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None, 'enum_strs': ['OneShot', 'AutoCount']}), ('scaler_delay', {'source': 'PV:gov:scaler1.DLY', 'dtype': 'number', 'shape': [], 'precision': 3, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_auto_count_delay', {'source': 'PV:gov:scaler1.DLY1', 'dtype': 'number', 'shape': [], 'precision': 3, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_names_name1', {'source': 'PV:gov:scaler1.NM1', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name2', {'source': 'PV:gov:scaler1.NM2', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name3', {'source': 'PV:gov:scaler1.NM3', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name4', {'source': 'PV:gov:scaler1.NM4', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name5', {'source': 'PV:gov:scaler1.NM5', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name6', {'source': 'PV:gov:scaler1.NM6', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name7', {'source': 'PV:gov:scaler1.NM7', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name8', {'source': 'PV:gov:scaler1.NM8', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name9', {'source': 'PV:gov:scaler1.NM9', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name10', {'source': 'PV:gov:scaler1.NM10', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name11', {'source': 'PV:gov:scaler1.NM11', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name12', {'source': 'PV:gov:scaler1.NM12', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name13', {'source': 'PV:gov:scaler1.NM13', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name14', {'source': 'PV:gov:scaler1.NM14', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name15', {'source': 'PV:gov:scaler1.NM15', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name16', {'source': 'PV:gov:scaler1.NM16', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name17', {'source': 'PV:gov:scaler1.NM17', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name18', {'source': 'PV:gov:scaler1.NM18', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name19', {'source': 'PV:gov:scaler1.NM19', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name20', {'source': 'PV:gov:scaler1.NM20', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name21', {'source': 'PV:gov:scaler1.NM21', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name22', {'source': 'PV:gov:scaler1.NM22', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name23', {'source': 'PV:gov:scaler1.NM23', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name24', {'source': 'PV:gov:scaler1.NM24', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name25', {'source': 'PV:gov:scaler1.NM25', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name26', {'source': 'PV:gov:scaler1.NM26', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name27', {'source': 'PV:gov:scaler1.NM27', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name28', {'source': 'PV:gov:scaler1.NM28', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name29', {'source': 'PV:gov:scaler1.NM29', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name30', {'source': 'PV:gov:scaler1.NM30', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name31', {'source': 'PV:gov:scaler1.NM31', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_names_name32', {'source': 'PV:gov:scaler1.NM32', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None}), ('scaler_time', {'source': 'PV:gov:scaler1.T', 'dtype': 'number', 'shape': [], 'precision': 3, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_freq', {'source': 'PV:gov:scaler1.FREQ', 'dtype': 'number', 'shape': [], 'precision': 3, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_preset_time', {'source': 'PV:gov:scaler1.TP', 'dtype': 'number', 'shape': [], 'precision': 3, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_auto_count_time', {'source': 'PV:gov:scaler1.TP1', 'dtype': 'number', 'shape': [], 'precision': 3, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_update_rate', {'source': 'PV:gov:scaler1.RATE', 'dtype': 'number', 'shape': [], 'precision': 3, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_auto_count_update_rate', {'source': 'PV:gov:scaler1.RAT1', 'dtype': 'number', 'shape': [], 'precision': 3, 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0}), ('scaler_egu', {'source': 'PV:gov:scaler1.EGU', 'dtype': 'string', 'shape': [], 'units': None, 'lower_ctrl_limit': None, 'upper_ctrl_limit': None})])}}
	 name primary
	 hints {'scaler': {'fields': ['scaler_channels_chan1', 'scaler_channels_chan2', 'scaler_channels_chan3', 'scaler_channels_chan4', 'scaler_channels_chan7']}}
	 object_keys {'scaler': ['scaler_channels_chan1', 'scaler_channels_chan2', 'scaler_channels_chan3', 'scaler_channels_chan4', 'scaler_channels_chan7']}
~~~~~~~~~~~~~~~~~
event 7
	 descriptor cd11f35c-0811-4533-ab72-68f68d950fa0
	 time 1532469887.8352284
	 data {'scaler_channels_chan1': 16000000.0, 'scaler_channels_chan2': 7.0, 'scaler_channels_chan3': 8.0, 'scaler_channels_chan4': 9.0, 'scaler_channels_chan7': 8.0}
	 timestamps {'scaler_channels_chan1': 1532469887.717194, 'scaler_channels_chan2': 1532469887.717194, 'scaler_channels_chan3': 1532469887.717194, 'scaler_channels_chan4': 1532469887.717194, 'scaler_channels_chan7': 1532469887.717194}
	 seq_num 1
	 uid 6c55bd62-e35f-43e2-9227-6378a89bb697
	 filled {}
~~~~~~~~~~~~~~~~~
event 7
	 descriptor cd11f35c-0811-4533-ab72-68f68d950fa0
	 time 1532469889.5566924
	 data {'scaler_channels_chan1': 16000000.0, 'scaler_channels_chan2': 7.0, 'scaler_channels_chan3': 6.0, 'scaler_channels_chan4': 9.0, 'scaler_channels_chan7': 9.0}
	 timestamps {'scaler_channels_chan1': 1532469889.551645, 'scaler_channels_chan2': 1532469889.551645, 'scaler_channels_chan3': 1532469889.551645, 'scaler_channels_chan4': 1532469889.551645, 'scaler_channels_chan7': 1532469889.551645}
	 seq_num 2
	 uid 3ad0f681-fecb-4991-9f98-fea36072ea12
	 filled {}
~~~~~~~~~~~~~~~~~
event 7
	 descriptor cd11f35c-0811-4533-ab72-68f68d950fa0
	 time 1532469891.087019
	 data {'scaler_channels_chan1': 16000000.0, 'scaler_channels_chan2': 7.0, 'scaler_channels_chan3': 9.0, 'scaler_channels_chan4': 8.0, 'scaler_channels_chan7': 9.0}
	 timestamps {'scaler_channels_chan1': 1532469891.081951, 'scaler_channels_chan2': 1532469891.081951, 'scaler_channels_chan3': 1532469891.081951, 'scaler_channels_chan4': 1532469891.081951, 'scaler_channels_chan7': 1532469891.081951}
	 seq_num 3
	 uid cf614639-d995-46be-9fe1-0e5e20f951b0
	 filled {}
~~~~~~~~~~~~~~~~~
stop 6
	 run_start 38f77323-57e9-443e-8114-871f85347610
	 time 1532469891.0885608
	 uid 5e9d43f1-7ff7-4258-9552-b649272125db
	 exit_status success
	 reason 
	 num_events {'primary': 3}
~~~~~~~~~~~~~~~~~
('38f77323-57e9-443e-8114-871f85347610',)
```


## Quit the jupyter notebook

* Save the "lesson1" page if you wish.
* Press the **Logout** button in the upper right corner of "lesson1" and close the page.
* Press the **Logout** button in the upper right corner of "Home" and close the page.
* Type Control-C (^C) twice in the jupyter-notebook console to kill the server.